In [1]:
import pandas as pd
import numpy as np

from itertools import product

In [4]:
1+2j + 3+5j

(4+7j)

In [2]:
np.exp(-3+10j)

(-0.041774911583657474-0.027085216241410345j)

In [4]:
def solutions_ad_mod_c(c,r=1):
    # it is unique
    assert r<c
    return [[a,d] for a,d in product(range(1,c),range(1,c)) if (a*d)%c==r]

def divisors(g):
    divs = []
    for s in range(1,int(np.sqrt(g))+1):
        if g%s == 0:
            divs.append(s)
            divs.append(g//s)
    return list(set(divs))

def abs_norm(c):
    return np.sqrt(np.real(np.conj(c) * c))

In [5]:
divisors(24)

[1, 2, 3, 4, 6, 8, 12, 24]

In [6]:
solutions_ad_mod_c(c=6,r=3)

[[1, 3], [3, 1], [3, 3], [3, 5], [5, 3]]

In [7]:
np.real(np.conj(1+1j) * (1+1j))

2.0

# Kloosterman Sums


In [8]:
def kloosterman(m,n,c):
    I = 1j
    Pi = np.pi
    sols = np.array(solutions_ad_mod_c(c,r=1))
    exponent = 2*Pi*I*(sols[:,0]*m/c + sols[:,1]*n/c)
    return np.exp(exponent).sum()

In [9]:
kloosterman(7,1,6)

(-1.0000000000000062+2.7755575615628914e-15j)

In [10]:
m = (3*5) * 100
n = (3*5)*12
c = (3*5)*123

k0 = kloosterman(m,n,c)
k0

(-13.319984562513362+2.007283228522283e-12j)

In [14]:
# selberg

def selberg_kloosterman(m,n,c,p=1):
    gcd = np.gcd.reduce([m,n,c])
    return sum(s* kloosterman(m*n//s**2,p,c//s) for s in divisors(gcd))

In [15]:
k1 = selberg_kloosterman(m,n,c)

In [16]:
abs_norm(k1-k0)

3.760908043175186e-09

# 2nd selberg Identity (Gomes 2017)

$$\sum_{d|(m,p,c)} K(n,mp/d^2,c/d) = \sum_{s|(m,n,c)} K(nm/s^2,p,c/s)$$

In [17]:
def selberg_kloost_2(m,n,c,p=1):
    gcd = np.gcd.reduce([m,p,c])
    Ks = 0
    for s in divisors(gcd):
            Ks += s* kloosterman(n,m*p//s**2,c//s)
    return sum(s* kloosterman(n,m*p//s**2,c//s) for s in divisors(gcd))


In [18]:
selberg_kloosterman(m,n,c,25)

(88.78924333197843-8.264446904604483e-10j)

In [19]:
selberg_kloost_2(m,n,c,p=25)

(88.78924333087976-4.0431435976984176e-10j)

# generalized Kloosterman Sums

## multiplier matrix

In [45]:
def multiplier_matrix(i,j,k,a,d,c):
    I = 1j
    M = 0
    Pi = np.pi
    m = np.arange(c)
    exponent = (Pi*I*a)/(2*k*c)*(j+2*k*m)**2 - (Pi*I)/(k*c)*(j+2*k*m)*i + (Pi*I*d)/(2*k*c)*i**2
    M =  np.exp(exponent).sum()
    return M/np.sqrt(2*I*k*c)

In [48]:
def gamma_matrix(a,d,c, b=None):
    if c!=0:
        assert a*d%c == 1
        b = (a*d-1)//c
    else:
        assert d*a==1
        assert b is not None
    return np.array([[a,b],[c,d]], dtype=int)



def spectral_matrix(level,gamma):
    c = gamma[1,0]
    a = gamma[0,0]%c
    d = gamma[1,1]%c
    vals = np.array([multiplier_matrix(i,j,level,a,d,c) \
                    for i,j in product(range(1,2*level+1), range(1,2*level+1))])
    return vals.reshape(2*level, 2*level).T

In [56]:
gamma_0 = gamma_matrix(2,3,5)
gamma_1 = gamma_matrix(1,1,5)
gamma_01 = np.matmul(gamma_0,gamma_1)

In [62]:
gamma_0

array([[2, 1],
       [5, 3]])

In [61]:
gamma_01

array([[ 7,  1],
       [20,  3]])

In [63]:
M0 = spectral_matrix(5,gamma_0)
M1 = spectral_matrix(5,gamma_1)
M01 = spectral_matrix(5,gamma_01)

In [66]:
M01.max()

(0.8090169943749971+0.5877852522924583j)

In [69]:
M01_ = np.matmul(M0,M1)

In [71]:
diff = (M01 - M01_)
np.sqrt((diff*np.conjugate(diff))).max()

(1.0438090993682666e-13+0j)

## generalized sums

In [312]:
def modified_kloosterman(n,i,p,j,k,c,r):
    I = 1j
    Pi = np.pi
    sols = np.array(solutions_ad_mod_c(c,r))
    m_ij = np.array([multiplier_matrix(i,j,k,a,d,c) for a,d in sols])
    exponent = 2*Pi*I*(sols[:,1]/c)*(n - i**2/(4*k)) + 2*Pi*I*(sols[:,0]/c)*(p - j**2/(4*k)) 
    return (np.exp(exponent)*m_ij).sum()

def selberg_gen_kloosterman(n,i,p,j,k,c,r):
    gcd = np.gcd.reduce([c,r,n,i])
    k_ = [s**(3/2) * modified_kloosterman(n*r//s**2,i//s,p,j,k,c//s,1) for s in divisors(gcd)]
    print(k_)
    K = [ kl for kl in k_]
    return np.array(K).sum()

def selberg_gen_kloosterman_2(n,i,p,j,k,c,r):
    gcd = np.gcd.reduce([c,r])
    K = 0
    for s in divisors(gcd):
        orbits = [(p+j*m+k*m**2,j+2*k*m) for m in range(s) if (p+j*m+k*m**2)%s == 0]
        print(len(orbits))
        K += sum( [(r*s)**(1/2)*modified_kloosterman(n,i,np*r//s**2,lp*r//s,k*r,c//s,1) for np,lp in orbits ])
    return K

In [322]:
data = {'n' : 4,
        'k' : 3,
        'p' : 3,
        'i' : 2,
        'j' : 0,
        'c' : 4,
        'r' : 2
}
data

{'n': 4, 'k': 3, 'p': 3, 'i': 2, 'j': 0, 'c': 4, 'r': 2}

In [323]:
modified_kloosterman(**data)

(-0.5773502691896231+0.5773502691896274j)

In [324]:
selberg_gen_kloosterman(**data)

[(0.5773502691896253-0.5773502691896281j), (-1.154700538379251+1.1547005383792526j)]


(-0.5773502691896257+0.5773502691896245j)

In [325]:
selberg_gen_kloosterman_2(**data)

1
1


(-0.577350269189618+0.5773502691896337j)